## **Extract features from image**

### **Import necessary libraries**

In [2]:
import os 
import sys

import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 

import warnings 
warnings.filterwarnings('ignore')

from PIL import Image 

import torch
import torchvision 
import torchvision.transforms as transforms 
import torchvision.models as models 
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
from torch.autograd import Variable


In [3]:
current_cwd = os.getcwd()
lst = current_cwd.split('\\', 2)
new_cwd = '/'.join(lst[:2])
os.chdir(new_cwd)
sys.path.append(new_cwd + '/code')

In [4]:
IMAGES_PATH = 'data/flickr8k/Images'
CAPTIONS_PATH = 'data/flickr8k/captions.txt'

data = pd.read_csv(CAPTIONS_PATH)
data

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...
...,...,...
40450,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face
40451,997722733_0cb5439472.jpg,A man is rock climbing high in the air .
40452,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...
40453,997722733_0cb5439472.jpg,A rock climber in a red shirt .


In [9]:
images_data = data.drop_duplicates(subset=['image'])
images_data

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
5,1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
10,1002674143_1b742ab4b8.jpg,A little girl covered in paint sits in front o...
15,1003163366_44323f5815.jpg,A man lays on a bench while his dog sits by him .
20,1007129816_e794419615.jpg,A man in an orange hat starring at something .
...,...,...
40430,990890291_afc72be141.jpg,A man does a wheelie on his bicycle on the sid...
40435,99171998_7cc800ceef.jpg,A group is sitting around a snowy crevasse .
40440,99679241_adc853a5c0.jpg,A grey bird stands majestically on a beach whi...
40445,997338199_7343367d7f.jpg,A person stands near golden walls .


### **Load the AlexNet**

In [5]:
alexnet = models.alexnet(pretrained=True)

alexnet.eval()

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [02:14<00:00, 1.81MB/s] 


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
# Use the model obeject to select the desired layer
layer = alexnet.classifier[-2]

In [7]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [8]:
def get_vector_alexnet(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 4096
    my_embedding = torch.zeros(4096)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    alexnet(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [10]:
alexnet_image = {}
cnt = 0
for img in images_data.image:
    alexnet_image[img] = get_vector_alexnet(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [11]:
alexnet_features = images_data

In [12]:
alexnet_features['image features'] = alexnet_features.image.map(alexnet_image)

In [13]:
alexnet_features = alexnet_features[['image', 'image features']].reindex()
with open('Image features/alexnet.csv', mode='w', encoding='utf-8') as file: 
    alexnet_features.to_csv(file, index=False)